In [50]:
from ecpy.curves import Curve, Point, ECPyException
import os
import struct

# Useful Functions

Below is a list of useful function, these can be ignored as they are not essential to understand the protocol.

In [17]:
def uint256_to_str(s):
    """Convert bytes to uint256"""
    assert 0 <= s < 2**256
    t = []
    for i in range(8):
        t.append((s >> (i * 32) & 0xffffffff))
    s = struct.pack(b"<IIIIIIII", *t)
    return s

def uint256_from_str(s):
    """Convert bytes to uint256"""
    r = 0
    t = struct.unpack(b"<IIIIIIII", s[:32])
    for i in range(8):
        r += t[i] << (i * 32)
    return r

def make_random_point(curve, rnd_bytes=os.urandom):
    while True:
        x = uint256_from_str(rnd_bytes(32))
        y = cv.x_recover(x)
        try: point = Point(x, y, curve)
        except ECPyException: continue
        break
    
    if (rnd_bytes(1)) % 2 == 0:
        point.y = -point.y
        
    return point

# Elliptic Curve Parameters

Here is a list of the elliptic curve parameters used in this demonstration, along with datatype as well.

In [33]:
curve = Curve.get_curve('secp256k1')
G = curve.generator
field = curve.field
order = curve.order

assert type(order) is int
assert type(field) is int
assert type(G) is Point

# Public Key/Private Key

Here is the step where we generate the public and private key. We will make the following assumption.

Even if you know what $G$ and $A$ are, there exists no polynomial time algorithm to compute $a$ (Discrete log assumption).

Alternatively the following lines can be written like this.

$a \in F_p$

$A = g^a$

In [51]:
# a is our private key
a = uint256_from_str(os.urandom(32))
A = a*G
assert A == a*G
print(A/G) # This will fail however it shows that you can't just divide A and G to get back a.

TypeError: unsupported operand type(s) for /: 'Point' and 'Point'

# Schnorr Protocol

We will be using a varient of the Schnorr Protocol, however it is important to understand this example.

In [32]:
## TAG

# Secret: a, t
# Public: T

# Step 1 create blinding factor
t = uint256_from_str(os.urandom(32)) % order

# Commit the value t and send the result to the reader.
T = t*G

In [31]:
## READER
# Receives T from the tag above.
# Computes a challenge c and sends this off the reader.
c = uint256_from_str(os.urandom(32)) % order

In [30]:
## TAG
# Secret: a, t
# Compute s and send this to the reader
s = (a*c + t) % order

In [29]:
## READER
# Check s
assert s*G == c*A + T

## Why This is Correct

Okay so given $T=g^t$, $c$, and $A=g^a$ how do we know that the $s$ is correct.

$\begin{align*} g^s &= g^{a*c + t} \\ 
                    &= g^{a*c} g^{t} \\
                    &= A^{c}T\end{align*}$
                    
Keep in mind that $c$, $A$ and $T$ are public. Furthermore, as long as $c$ and $t$ is randomly selected, then we cannot reconstruct $a$.

#### Modified Schnorr Protocol (Algorithm we propose to use)

Long story short, this is really similar to the first example. The main difference is that in the first example, you have to assume that the reader sends you random challenges or else your secret can be reveals. In these scheme, you don't have the trust that the reader is honest.

In [52]:
## TAG

# Step 1 create blinding factor
r = uint256_from_str(os.urandom(32)) % order

# commitment
R = r*G

# Send commitment to Reader

In [42]:
# READER

# Reader creates a secret
y = uint256_from_str(os.urandom(32)) % order

# reader's public key
Y = y*G

# make challenge and send to tag
e = uint256_from_str(os.urandom(32)) % order

In [44]:
# TAG
d = (r*Y).x
s = (d + a + e*r) % order
# send s

In [45]:
# READER
d_prime = (y*R).x
assert s*G - d_prime*G - e*R == A